In [1]:
# Importing Libraries
import pandas as pd
import glob ##For reading multiple files
import numpy as np
import dask.dataframe as dd


In [2]:
folder_path = 'NJ-citibike-tripdata/*.csv' ##Path to the folder containing the data files
csv_files = glob.glob(folder_path) ##Reading all the files in the folder
csv_files = csv_files[:] ##Reading only the first :5 files
data_df = pd.concat(
    [pd.read_csv(file, dtype={5: str, 7: str,}) for file in csv_files]
)  ##Concatenating all the files to form a
data_df.count()

ride_id               1103062
rideable_type         1103062
started_at            1103062
ended_at              1103062
start_station_name    1102958
start_station_id      1102958
end_station_name      1099894
end_station_id        1099592
start_lat             1103062
start_lng             1103062
end_lat               1102690
end_lng               1102690
member_casual         1103062
dtype: int64

In [47]:
data_df = data_df.dropna(subset=['end_lat'])
data_df = data_df.dropna(subset=["start_lat"])
data_df = data_df.dropna(subset=["start_station_id"])
data_df = data_df.dropna(subset=["end_station_id"])
data_df['start_station_name'] = data_df['start_station_name'].fillna('unknown')
data_df['end_station_name'] = data_df['end_station_name'].fillna('unknown')
data_df.columns = data_df.columns.str.upper()  ##Converting column names to uppercase

data_df.count()

RIDE_ID               1099534
RIDEABLE_TYPE         1099534
STARTED_AT            1099534
ENDED_AT              1099534
START_STATION_NAME    1099534
START_STATION_ID      1099534
END_STATION_NAME      1099534
END_STATION_ID        1099534
START_LAT             1099534
START_LNG             1099534
END_LAT               1099534
END_LNG               1099534
MEMBER_CASUAL         1099534
dtype: int64

In [48]:
data_df["STARTED_AT"].min(), data_df["STARTED_AT"].max()

('2024-01-01 00:06:21', '2025-01-31 23:53:42.476')

###Data Exploration & Pre-Processing


In [49]:

data_df.isnull().sum() ##Checking for missing values
data_df = data_df.dropna() ##Dropping missing values
data_df = data_df.drop_duplicates() ##Dropping duplicates


In [50]:
##Renaming columns
data_df = data_df.rename(columns={'RIDEABLE_TYPE': 'BIKE_TYPE', 'START_STATION_NAME': 'START_STATION',
                                  'END_STATION_NAME': 'END_STATION','STARTED_AT':'START_DT','ENDED_AT':'END_DT'}) 
data_df["END_DT"] = pd.to_datetime(data_df["END_DT"], format="ISO8601")
data_df["START_DT"] = pd.to_datetime(data_df["START_DT"], format="ISO8601")
data_df.head()

,RIDE_ID,BIKE_TYPE,START_DT,END_DT,START_STATION,START_STATION_ID,END_STATION,END_STATION_ID,START_LAT,START_LNG,END_LAT,END_LNG,MEMBER_CASUAL
0,0744109F13385D1D,electric_bike,2024-01-15 15:18:07,2024-01-15 15:32:44,Morris Canal,JC072,Oakland Ave,JC022,40.712297,-74.038185,40.737604,-74.052478,member
1,B1488BFEF9118000,classic_bike,2024-01-13 15:32:50,2024-01-13 15:36:18,JC Medical Center,JC110,Grove St PATH,JC115,40.715391,-74.049692,40.719410,-74.043090,member
2,95A2FE8E51B4C836,classic_bike,2024-01-19 13:11:00,2024-01-19 13:14:44,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,40.716366,-74.034344,member
3,95D9AFF6A1652DC1,classic_bike,2024-01-23 07:03:49,2024-01-23 07:07:11,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,40.716366,-74.034344,member
4,5F7408988A83B1B3,classic_bike,2024-01-01 16:46:10,2024-01-01 16:50:31,Morris Canal,JC072,Harborside,JC104,40.712419,-74.038526,40.719252,-74.034234,member


In [51]:

# Created function to calculate distance using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    r = 3958.8  # Radius of Earth in miles
    return r * c

# Apply the function to each row in the DataFrame
data_df['DISTANCE_MILES'] = data_df.apply(
    lambda row: haversine(row['START_LAT'], row['START_LNG'],
                          row['END_LAT'], row['END_LNG']), axis=1
)

data_df['DISTANCE_MILES'] = data_df['DISTANCE_MILES'].round(2) ##Rounding distance to 2 decimal places

In [52]:
# Calculating trips total duration in  mins & seconds
total_seconds = (data_df['END_DT'] - data_df['START_DT']).dt.total_seconds()
data_df['DURATION_MM_SS'] = (total_seconds // 60).astype(int).astype(str).str.zfill(2) + ':' + \
                            (total_seconds % 60).astype(int).astype(str).str.zfill(2)

# Finding AVG MPH
duration_hours = total_seconds / 3600
# Estimate distance travelled for round trips -- 8.5 mph is the average speed for a city bike
#data_df = data_df.reset_index(drop=True)

# data_df.loc[
#     (data_df["START_STATION"] == data_df["END_STATION"]),
#     "DISTANCE_MILES",
# ] = (
#     duration_hours * 8.5
# )
# Calculate average speed
data_df['AVG_MPH'] = data_df['DISTANCE_MILES'] / duration_hours



In [53]:
data_df[(data_df['START_STATION'] == data_df['END_STATION']) & (data_df['DISTANCE_MILES'] == 0.00)] ##Checking for round trips  


,RIDE_ID,BIKE_TYPE,START_DT,END_DT,START_STATION,START_STATION_ID,END_STATION,END_STATION_ID,START_LAT,START_LNG,END_LAT,END_LNG,MEMBER_CASUAL,DISTANCE_MILES,DURATION_MM_SS,AVG_MPH
23,DD1599465F8062F0,classic_bike,2024-01-11 22:26:58.000,2024-01-11 22:29:12.000,Jersey & 3rd,JC074,Jersey & 3rd,JC074,40.723332,-74.045953,40.723332,-74.045953,member,0.0,02:14,0.0
25,5E9C9A78F95893B2,electric_bike,2024-01-14 15:33:42.000,2024-01-14 16:00:14.000,Jersey & 3rd,JC074,Jersey & 3rd,JC074,40.723332,-74.045953,40.723332,-74.045953,member,0.0,26:32,0.0
49,18DFA2D71CFD417D,classic_bike,2024-01-18 15:07:23.000,2024-01-18 15:25:13.000,Church Sq Park - 5 St & Park Ave,HB601,Church Sq Park - 5 St & Park Ave,HB601,40.742659,-74.032233,40.742659,-74.032233,member,0.0,17:50,0.0
50,D3453B7C2E2947B6,electric_bike,2024-01-16 07:21:14.000,2024-01-16 07:21:59.000,Church Sq Park - 5 St & Park Ave,HB601,Church Sq Park - 5 St & Park Ave,HB601,40.742671,-74.032321,40.742659,-74.032233,member,0.0,00:45,0.0
86,C93FBD84749ECBC9,electric_bike,2024-01-01 10:43:45.000,2024-01-01 10:47:20.000,Baldwin at Montgomery,JC020,Baldwin at Montgomery,JC020,40.723659,-74.064194,40.723659,-74.064194,member,0.0,03:35,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50529,45FA38B5F5D31BB4,electric_bike,2025-01-23 14:01:44.747,2025-01-23 14:41:10.531,Baldwin at Montgomery,JC020,Baldwin at Montgomery,JC020,40.723659,-74.064194,40.723659,-74.064194,casual,0.0,39:25,0.0
50530,E272FA6257F7811C,electric_bike,2025-01-11 20:12:19.620,2025-01-11 20:22:22.453,Baldwin at Montgomery,JC020,Baldwin at Montgomery,JC020,40.723659,-74.064194,40.723659,-74.064194,member,0.0,10:02,0.0
50531,D89117FDD781C2B9,electric_bike,2025-01-24 20:01:10.214,2025-01-24 20:05:15.923,Baldwin at Montgomery,JC020,Baldwin at Montgomery,JC020,40.723659,-74.064194,40.723659,-74.064194,casual,0.0,04:05,0.0
50532,4DFB053EC0282A47,electric_bike,2025-01-30 16:07:59.308,2025-01-30 16:15:55.623,Baldwin at Montgomery,JC020,Baldwin at Montgomery,JC020,40.723659,-74.064194,40.723659,-74.064194,casual,0.0,07:56,0.0


In [54]:
data_df[
    (data_df["START_DT"] != data_df["END_DT"]) & (data_df["DISTANCE_MILES"] == 0)
]  ##Checking for trips that started and ended on different days

,RIDE_ID,BIKE_TYPE,START_DT,END_DT,START_STATION,START_STATION_ID,END_STATION,END_STATION_ID,START_LAT,START_LNG,END_LAT,END_LNG,MEMBER_CASUAL,DISTANCE_MILES,DURATION_MM_SS,AVG_MPH
23,DD1599465F8062F0,classic_bike,2024-01-11 22:26:58.000,2024-01-11 22:29:12.000,Jersey & 3rd,JC074,Jersey & 3rd,JC074,40.723332,-74.045953,40.723332,-74.045953,member,0.0,02:14,0.0
25,5E9C9A78F95893B2,electric_bike,2024-01-14 15:33:42.000,2024-01-14 16:00:14.000,Jersey & 3rd,JC074,Jersey & 3rd,JC074,40.723332,-74.045953,40.723332,-74.045953,member,0.0,26:32,0.0
49,18DFA2D71CFD417D,classic_bike,2024-01-18 15:07:23.000,2024-01-18 15:25:13.000,Church Sq Park - 5 St & Park Ave,HB601,Church Sq Park - 5 St & Park Ave,HB601,40.742659,-74.032233,40.742659,-74.032233,member,0.0,17:50,0.0
50,D3453B7C2E2947B6,electric_bike,2024-01-16 07:21:14.000,2024-01-16 07:21:59.000,Church Sq Park - 5 St & Park Ave,HB601,Church Sq Park - 5 St & Park Ave,HB601,40.742671,-74.032321,40.742659,-74.032233,member,0.0,00:45,0.0
86,C93FBD84749ECBC9,electric_bike,2024-01-01 10:43:45.000,2024-01-01 10:47:20.000,Baldwin at Montgomery,JC020,Baldwin at Montgomery,JC020,40.723659,-74.064194,40.723659,-74.064194,member,0.0,03:35,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50529,45FA38B5F5D31BB4,electric_bike,2025-01-23 14:01:44.747,2025-01-23 14:41:10.531,Baldwin at Montgomery,JC020,Baldwin at Montgomery,JC020,40.723659,-74.064194,40.723659,-74.064194,casual,0.0,39:25,0.0
50530,E272FA6257F7811C,electric_bike,2025-01-11 20:12:19.620,2025-01-11 20:22:22.453,Baldwin at Montgomery,JC020,Baldwin at Montgomery,JC020,40.723659,-74.064194,40.723659,-74.064194,member,0.0,10:02,0.0
50531,D89117FDD781C2B9,electric_bike,2025-01-24 20:01:10.214,2025-01-24 20:05:15.923,Baldwin at Montgomery,JC020,Baldwin at Montgomery,JC020,40.723659,-74.064194,40.723659,-74.064194,casual,0.0,04:05,0.0
50532,4DFB053EC0282A47,electric_bike,2025-01-30 16:07:59.308,2025-01-30 16:15:55.623,Baldwin at Montgomery,JC020,Baldwin at Montgomery,JC020,40.723659,-74.064194,40.723659,-74.064194,casual,0.0,07:56,0.0


In [55]:
#data_df['DAY_OF_WEEK'] = data_df['START_DT'].dt.day_name() ##Extracting day of the week 

In [56]:
#Creating a function to get the time of the day
def time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning(5am-12pm)'
    elif 12 <= hour < 17:
        return 'Afternoon(12pm-5pm)'
    elif 17 <= hour < 21:
        return 'Evening(5pm-9pm)'
    else:
        return 'Night(9pm-5am)'
data_df['TIME_OF_DAY'] = data_df['START_DT'].dt.hour.apply(time_of_day) ##Applying the function to get the time of the day

In [57]:
# Define a function to classify the season based on the month
def classify_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Fall"


# Extract the month from the start_dt column
data_df["start_month"] = pd.to_datetime(data_df["START_DT"]).dt.month

# Apply the classify_season function to create the season column
data_df["SEASON"] = data_df["start_month"].apply(classify_season)

# Drop the start_month column if not needed
data_df = data_df.drop("start_month", axis=1)


In [58]:
data_df['START_DT'] = data_df['START_DT'].dt.strftime('%Y-%m-%d') ##Converting start date to string
data_df['END_DT'] = data_df['END_DT'].dt.strftime('%Y-%m-%d') ##Converting end date to string
data_df = data_df.drop(
    ["START_STATION_ID", "END_STATION_ID"], axis=1
)  ##Dropping columns
#Formating the bike type for visualization
data_df["BIKE_TYPE"] = data_df["BIKE_TYPE"].str[:-5]

data_df

,RIDE_ID,BIKE_TYPE,START_DT,END_DT,START_STATION,END_STATION,START_LAT,START_LNG,END_LAT,END_LNG,MEMBER_CASUAL,DISTANCE_MILES,DURATION_MM_SS,AVG_MPH,TIME_OF_DAY,SEASON
0,0744109F13385D1D,electric,2024-01-15,2024-01-15,Morris Canal,Oakland Ave,40.712297,-74.038185,40.737604,-74.052478,member,1.90,14:37,7.799316,Afternoon(12pm-5pm),Winter
1,B1488BFEF9118000,classic,2024-01-13,2024-01-13,JC Medical Center,Grove St PATH,40.715391,-74.049692,40.719410,-74.043090,member,0.44,03:28,7.615385,Afternoon(12pm-5pm),Winter
2,95A2FE8E51B4C836,classic,2024-01-19,2024-01-19,Morris Canal,Exchange Pl,40.712419,-74.038526,40.716366,-74.034344,member,0.35,03:44,5.625000,Afternoon(12pm-5pm),Winter
3,95D9AFF6A1652DC1,classic,2024-01-23,2024-01-23,Morris Canal,Exchange Pl,40.712419,-74.038526,40.716366,-74.034344,member,0.35,03:22,6.237624,Morning(5am-12pm),Winter
4,5F7408988A83B1B3,classic,2024-01-01,2024-01-01,Morris Canal,Harborside,40.712419,-74.038526,40.719252,-74.034234,member,0.52,04:21,7.172414,Afternoon(12pm-5pm),Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50606,7ADDE3DF87904CD3,electric,2025-01-03,2025-01-03,Baldwin at Montgomery,City Hall,40.723659,-74.064194,40.717732,-74.043845,member,1.14,05:30,12.420105,Morning(5am-12pm),Winter
50607,0729D9E8BC06E6B1,electric,2025-01-15,2025-01-15,Baldwin at Montgomery,City Hall,40.723659,-74.064194,40.717732,-74.043845,member,1.14,06:42,10.202458,Evening(5pm-9pm),Winter
50608,E5AE184A09BD3541,electric,2025-01-07,2025-01-07,Baldwin at Montgomery,City Hall,40.723659,-74.064194,40.717732,-74.043845,member,1.14,07:53,8.669348,Evening(5pm-9pm),Winter
50609,958ED54D039E3F0A,classic,2025-01-23,2025-01-23,Baldwin at Montgomery,City Hall,40.723659,-74.064194,40.717732,-74.043845,member,1.14,09:22,7.300439,Afternoon(12pm-5pm),Winter


In [59]:
print(data_df['DISTANCE_MILES'].sum()) ##Total distance travelled



817750.34


In [44]:
# aggregating the data
# aggregated_df = data_df.groupby(['BIKE_TYPE', 'START_DT', 'START_STATION', 'MEMBER_CASUAL', 'TIME_OF_DAY']).agg({
#     'RIDE_ID': 'count',
#     'DISTANCE_MILES': 'sum',
#     'DURATION_MM_SS': 'sum',
#     'AVG_MPH': 'mean'
# }).reset_index()

# aggregated_df.columns = ['BIKE_TYPE', 'START_DT', 'START_STATION', 'MEMBER_CASUAL', 'TIME_OF_DAY', 'RIDE_COUNT', 'TOTAL_DISTANCE', 'TOTAL_DURATION', 'AVG_MPH']

# aggregated_df


In [60]:
# data_df_sample = data_df#.sample(n=900000, random_state=42)
# data_df_sample.to_csv('citibike_data.csv', index=False) ##Saving the sampled data to a csv file
import os
folder_path = r"Dashboard_data"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
chunk_size = 1000000  # Adjust as needed
for i, start in enumerate(range(0, data_df.shape[0], chunk_size)):
    chunk = data_df.iloc[start : start + chunk_size]
    chunk.to_csv(f"{folder_path}\citibike_data{i}.csv", index=False)